<a href="https://colab.research.google.com/github/kznts9v-1lya/formal-lang-course/blob/task5-rpq-research/researches/rpq_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Автор [kznts9v_1lya](https://github.com/kznts9v-1lya).

---

# Экспериментальное исследование производительности CPU и GPU версий алгоритма Regular Path Quering.

---

## Введение

Имеется **размеченный граф** $G$, задающий регулярный язык $L_G$.

Имеется **регулярное выражение** $Q$, задающее регулярный язык ограничений $L_Q$.

Задача $Regular Path Quering$ решает проблему поиска путей в заданных ограничениях. Они представляются в виде пар достижимых вершин графа, представляющего $L_G\cap L_Q$.

$RPQ = \{(v_i, v_j)|\exists\pi:w(v_i\pi v_j)\in L, v_i\in V_S, v_j\in V_F\}$, где $L = L_G\cap L_Q$

Известно свойство, что регулярные языки замкнуты относительно пересечения и по ним можно построить конечный автомат, допускающий эти языки. Именно поэтому $RPQ$ строит язык пересечения, задаваемый пересечением двух конечных автоматов $КА_{G}$ и $КА_{Q}$, допускающих $L_G$ и $L_Q$ соответсвенно, --- также конечным автоматом $КА_{I}$.

Обратимся к определению, $КА_3 = КА_1\cap КА_2 = (S^{1}\times S^{2}, \Delta^{3}, S^{1}_S\times S^{2}_S, S^{1}_F\times S^{2}_F)$, где функция переходов задаётся как
$\Delta^{3}: (v_i, v_j)\times l_{abel}\rightarrow (u_i, u_j)$

$
\begin{cases}
v_i\in S^{1}\times l\rightarrow u_i\in S^{1}\in\Delta^{1} \\
v_j\in S^{2}\times l\rightarrow u_j\in S^{2}\in\Delta^{2}
\end{cases}$

Для нахождения функции переходов $\Delta^{I}$ используется алгоритм, основанный на тензорном произведении булевых матриц смежности конечных автоматов $КА_{G}$ и $КА_{Q}$, также известном как произведение Кронекера, и последующем построении транизитивного замыкания.

В данной работе будет экспериментально исследована скорость работы описанного алгоритма решения проблемы $RPQ$, реализованный в двух вариантах --- на CPU и GPU.

---

## Постановка цели работы

Для достижения поставленной цели --- экспериментального исследования скорости работы описанного алгоритма решения проблемы $RPQ$ --- необходимо выполнить следующие подзадачи:

- Используя разряженные матрицы [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html) реализовать алгоритм пересечения двух конечных автоматов через тензорное произведение с использованием $CPU$;
- Используя библиотеку [puCuBool](https://pypi.org/project/pycubool/) реализовать алгоритм пересечения двух конечных автоматов через тензорное произведение с использованием технологии $CUDA$ для $GPU$;
- Сформировать датасет, необходимый для проведения экспериментов;
- Произвести сравнительный анализ производительности версий алгоритма.

---

## Исследуемые версии алгоритма

Элементами матрицы смежности для конечного автомата являются подмножества его меток, по которым возможен переход в данных состояниях. Чтобы не вводить операцию пересечения множеств в тензорном произведении, матрицы смежности двух конечных автоматов приводятся к виду булевых матриц по всем символам.

На практике, булевы матрицы являются сильно разреженными. Поэтому сравниваемые версии алгоритма используют библиотеки для работы с разреженными матрицами:

- [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)
- [puCuBool](https://pypi.org/project/pycubool/)

---

In [3]:
# Развёртывание https://github.com/kznts9v-1lya/formal-lang-course
!git clone https://github.com/kznts9v-1lya/formal-lang-course

fatal: destination path 'formal-lang-course' already exists and is not an empty directory.


In [4]:
# Установка внешних зависимостей
!pip install -r formal-lang-course/requirements.txt

In [6]:
# Инициализация директории в окружении
import sys
sys.path.insert(1, 'formal-lang-course')

---

## Описание данных для экспериментов

В качестве $L_G$ использовуются графы из [RDF](https://jetbrains-research.github.io/CFPQ_Data/dataset/RDF.html) датасета.

В запросах используются все общепринятые конструкции регулярных выражений (замыкание, конкатенация, альтернатива).

In [10]:
# Названия использующихся графов

GRAPHS = (
    # "skos",
    # "generations",
    # "travel",
    # "univ_bench",
    # "atom_primitive",
    # "biomedical_mesure_primitive",
    # "foaf",
    # "people_pets",
    # "funding",
    # "wine",
    # "pizza",
    # "core",
    # "pathways",
    # "enzyme",
    "eclass_514en",
    "go_hierarchy",
    "go",
    "geospecies",
)

In [13]:
# Информация о графах

from project.graph_tools import get_description

for graph in GRAPHS:
  print(f"{graph}: {get_description(graph)}")

eclass_514en: - number of nodes: 239111
	- number of edges: 360248
	- edge labels: {rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subPropertyOf'), rdflib.term.URIRef('http://purl.org/dc/elements/1.1/creator'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#imports'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://www.ebusiness-unibw.org/ontologies/eclass/5.1.4/#hierarchyCode')}
go_hierarchy: - number of nodes: 45007
	- number of edges: 490109
	- edge labels: {rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf')}
go: - number of nodes: 582929
	- number of edges: 1437437
	- e

file:///usr/local/lib/python3.7/dist-packages/cfpq_data/data/RDF/Graphs/<http:/sw.opencyc.org/concept/Mx4rvVi495wpEbGdrcN5Y29ycA> does not look like a valid URI, trying to serialize this will break.
file:///usr/local/lib/python3.7/dist-packages/cfpq_data/data/RDF/Graphs/<http:/sw.opencyc.org/concept/Mx4rvVi495wpEbGdrcN5Y29ycA> does not look like a valid URI, trying to serialize this will break.


geospecies: - number of nodes: 450609
	- number of edges: 2201532
	- edge labels: {rdflib.term.URIRef('http://rdf.geospecies.org/ont/geospecies#isExpectedIn'), rdflib.term.URIRef('http://rdf.geospecies.org/ont/geospecies#hasDateRange'), rdflib.term.URIRef('http://rdf.geospecies.org/ont/geospecies#hasWikispeciesArticle'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#seeAlso'), rdflib.term.URIRef('http://rdf.geospecies.org/ont/geospecies#hasLowExpectationOf'), rdflib.term.URIRef('http://rdf.geospecies.org/ont/geospecies#closeMatch'), rdflib.term.URIRef('http://rdf.geospecies.org/ont/geospecies#isUnknownAboutIn'), rdflib.term.URIRef('http://rdf.geospecies.org/ont/geospecies#hasExpectationOf'), rdflib.term.URIRef('http://rdf.geospecies.org/ont/geospecies#hasOmernik_3_Ecozone'), rdflib.term.URIRef('http://rdf.geospecies.org/ont/geospecies#hasSpecificEpithet'), rdflib.term.URIRef('http://purl.org/dc/terms/license'), rdflib.term.URIRef('http://rdf.geospecies.org/ont/geospecies#has

In [18]:
# Генерация регулярных выражений - запросов к графам

from pyformlang.regular_expression.regex_objects import Symbol
from pyformlang.regular_expression import Regex

def _regex_from_label(label):
  regex = Regex("")
  regex.head = Symbol(str(label))

  return regex

def query_one(labels):
  """
  (l0 | l1)* l2
  """

  label_regex_0 = _regex_from_label(labels[0])
  label_regex_1 = _regex_from_label(labels[1])
  label_regex_2 = _regex_from_label(labels[2])

  return label_regex_0.union(label_regex_1).kleene_star().concatenate(label_regex_2)

def query_two(labels):
  """
  l0 | l2 | l1*
  """

  label_regex_0 = _regex_from_label(labels[0])
  label_regex_1 = _regex_from_label(labels[1])
  label_regex_2 = _regex_from_label(labels[2])

  return label_regex_0.union(label_regex_2).union(label_regex_1.kleene_star())

def query_three(labels):
  """
  l0 l1 l2 (l3 | l1)*
  """ 

  label_regex_0 = _regex_from_label(labels[0])
  label_regex_1 = _regex_from_label(labels[1])
  label_regex_2 = _regex_from_label(labels[2])
  label_regex_3 = _regex_from_label(labels[3])

  return label_regex_0.concatenate(label_regex_1).concatenate(label_regex_2).\
  concatenate((label_regex_3.union(label_regex_1)).kleene_star())

def query_four(labels):
  """
  (l0 | l3)* | (l1 | l2)*
  """

  label_regex_0 = _regex_from_label(labels[0])
  label_regex_1 = _regex_from_label(labels[1])
  label_regex_2 = _regex_from_label(labels[2])
  label_regex_3 = _regex_from_label(labels[3])

  left_regex_part = (label_regex_0.union(label_regex_3)).kleene_star()
  right_regex_part = (label_regex_1.union(label_regex_2)).kleene_star()

  return left_regex_part.union(right_regex_part)